In [1]:
import pandas as pd
import json
import getpass
import requests
import datetime
import os


In [2]:
WB_TOKEN = os.getenv('WB_TOKEN')
headers = {'Authorization': WB_TOKEN}

In [3]:
subj_chars_url = 'https://content-api.wildberries.ru/content/v2/object/charcs/3756'
resp = requests.get(subj_chars_url, headers=headers)


In [4]:
d = json.loads( resp.text)

In [5]:
d

{'data': [{'charcID': 17596,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Материал изделия',
   'required': False,
   'unitName': '',
   'maxCount': 3,
   'popular': True,
   'charcType': 1},
  {'charcID': 88952,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Вес товара с упаковкой (г)',
   'required': False,
   'unitName': 'г',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90630,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Высота предмета',
   'required': False,
   'unitName': 'см',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90652,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Глубина предмета',
   'required': False,
   'unitName': 'см',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90673,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Ширина предмета',
   'requir

In [6]:
names = [n['name'] + "->" + str(n['required']) for n in d['data']]

In [12]:
import os
import re
import sqlite3

def gather_products_from_folder_and_db(db_path, images_folder):
    pattern = re.compile(r'^(\d+)-(\d+)\.jpg$', re.IGNORECASE)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    products_list = []

    for filename in os.listdir(images_folder):
        match = pattern.match(filename)
        if not match:
            continue

        product_id, quantityn_str = match.groups()
        cursor.execute("SELECT * FROM products WHERE product_id = ?", (product_id,))
        row = cursor.fetchone()

        if not row:
            continue

        (
            row_id, db_product_id, name, price_str, description, material,
            length_mm, width_mm, height_mkm, color, special,
            quantity_str_db, weight_g, transport_volume_m3, manufacturer_country
        ) = row

        height_cm = round(float(height_mkm) / 10000, 2) if height_mkm else None
        width_cm = round(float(width_mm) / 10, 2) if width_mm else None
        length_cm = round(float(length_mm) / 10, 2) if length_mm else None

        options = []

     
        options.append({
            "name": "Количество предметов в упаковке",
            "value": "100 шт",
            "is_variable": True,
            "charc_type": 1,
            "variable_values": ["100 шт"]
        })

        if color:
            options.append({
                "name": "Цвет",
                "value": color,
                "is_variable": True,
                "charc_type": 1,
                "variable_values": [color]
            })

        if material:
            options.append({
                "name": "Материал изделия",
                "value": material,
                "charc_type": 1
            })

        if height_cm:
            options.append({
                "name": "Высота предмета",
                "value": f"{height_cm} см",
                "charc_type": 4
            })

        if width_cm:
            options.append({
                "name": "Ширина предмета",
                "value": f"{width_cm} см",
                "charc_type": 4
            })

        options.append({
            "name": "Страна производства",
            "value": "Россия",
            "charc_type": 1
        })

        # if special:
        #     options.append({
        #         "name": "Комплектация",
        #         "value": ,
        #         "charc_type": 1
        #     })

        variant = {
            "vendorCode": f"zip_lock_{product_id}_100",
            "title": f"{name}",
            "description": description or "Описание отсутствует",
            "options": options,
        }

        product_object = {
            "subjectID": 3756,
            "variants": [variant]
        }

        products_list.append(product_object)

    conn.close()
    return products_list


In [13]:

db_file = "zip_lock_products_100.db"  # путь к вашей базе данных
folder_path = "zip_locks_images_900_1200"

result = gather_products_from_folder_and_db(db_file, folder_path)
for item in result:
    print(item)
# Здесь item — уже готовая структура для Wildberries API.
# Дальше можно вызывать свой метод create_product_cards(result), как в предыдущем примере.


{'subjectID': 3756, 'variants': [{'vendorCode': 'zip_lock_9604895_100', 'title': 'Пакеты Zip Lock, 150*200 мм, 70 мкм, 100 шт.', 'description': 'Пакеты Zip-Lock, прозрачного цвета с красной полосой, длиной 200 мм, шириной 150 мм, толщиной 70 мкм, в упаковке по 100 штук. Материал – полиэтилен высокого давления (ПВД). Производятся в России.\nОбласть применения:\nПакеты Зип-Лок применяются для упаковки продукции небольшого размера. Прозрачный пакет позволяет разглядеть содержимое, застежка zip-lock герметично закрывает пакет, обеспечивает сохранность продукта и возможность удобной транспортировки различных веществ. Используется при упаковке сыпучих товаров: чая, кофе, орехов, круп и мелких деталей.\nПреимущества:\nПрезентабельный внешний вид;\nПрочность;\nПростота и герметичность упаковки;\nНизкая цена.', 'options': [{'name': 'Количество предметов в упаковке', 'value': '100 шт', 'is_variable': True, 'charc_type': 1, 'variable_values': ['100 шт']}, {'name': 'Цвет', 'value': 'Прозрачный', '

In [14]:
API_URL = "https://content-api.wildberries.ru/content/v2/cards/upload"



def create_product_cards(products):
  
    try:
        response = requests.post(API_URL, json=products, headers=headers)
        response_data = response.json()
        
        if response.status_code == 200 and not response_data.get("error", True):
            print("Карточки товаров успешно созданы.")
        else:
            print(f"Ошибка: {response_data.get('errorText', 'Неизвестная ошибка')}")
            print("Дополнительные ошибки:", response_data.get("additionalErrors", {}))
        
        return response_data
    except requests.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

In [15]:
resp = create_product_cards(result)

Карточки товаров успешно созданы.
